In [1]:
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import csv
import sys
import os

In [2]:
remove_z = (os.environ.get('REMOVE_Z'))
remove_n = (os.environ.get('REMOVE_N'))

remove_n = int(float(remove_n.strip()))
remove_z = int(float(remove_z.strip()))

In [3]:
df = pd.read_csv('/home/submit/karich/NuCLR-MechInterp/data/ame2020.csv')

data_dict = {}
data_z_12_50 = {}
data_full = {}
data_all_z_12 = {}
for n,z,be,be_unc in zip(df.n,df.z,df.binding,df.binding_unc):
    if (be_unc * (z + n) > 100):
        continue
    if (n>20 and z>20):
        data_dict[(n,z)]=be
    if (z>=12 and z<=50):
        data_z_12_50[(n,z)]=be
    if (z>=12):
        data_all_z_12[(n,z)]=be
    data_full[(n,z)]=be
    
with open('/home/submit/karich/reranked_pcs_with_data.csv', newline='') as f:
    reader = csv.reader(f)
    full_features_Z = np.array([float(i) for i in next(reader)])
    full_features_N = np.array([float(i) for i in next(reader)])
    ouail_data = np.array([float(i[1:-1]) for i in next(reader)])
    full_pc0 = np.array([float(i[1:-1]) for i in next(reader)])
    full_pc2 = np.array([float(i[1:-1]) for i in next(reader)])
    full_pc1 = np.array([float(i[1:-1]) for i in next(reader)])

N = []
Z = []
pc0 = []
pc2 = []
pc1 = []
fixed_data = []

Z_z_12_50 = []
N_z_12_50 = []
fixed_data_z_12_50 = []

Z_full = []
N_full = []
fixed_data_full = []
pc0_full = []
pc1_full = []
pc2_full = []

Z_all_z_12 = []
N_all_z_12 = []
fixed_data_all_z_12 = []

for n, z, pc_0, d, pc_2, pc_1 in zip(full_features_N,full_features_Z,full_pc0, ouail_data, full_pc2, full_pc1):
    if (n,z) in data_dict.keys():
        N.append(n)
        Z.append(z)
        pc0.append(pc_0)
        fixed_data.append(d)
        pc2.append(pc_2)
        pc1.append(pc_1)
    if (n,z) in data_z_12_50.keys():
        N_z_12_50.append(n)
        Z_z_12_50.append(z)
        fixed_data_z_12_50.append(d)
    if (n,z) in data_full.keys():
        N_full.append(n)
        Z_full.append(z)
        fixed_data_full.append(d)
        pc0_full.append(pc_0)
        pc1_full.append(pc_1)
        pc2_full.append(pc_2)   
    if (n,z) in data_all_z_12.keys():
        Z_all_z_12.append(z)
        N_all_z_12.append(n)
        fixed_data_all_z_12.append(d)

N = np.array(N)
Z = np.array(Z)
pc0 = np.array(pc0)
pc2 = np.array(pc2)
pc1 = np.array(pc1)
pc0_over_1000000 = pc0/1000000
pc2_over_1000000 = pc2/1000000
pc1_over_1000000 = pc1/1000000
fixed_data = np.array(fixed_data)
fixed_data_over_1000000 = fixed_data/1000000

N_z_12_50 = np.array(N_z_12_50)
Z_z_12_50 = np.array(Z_z_12_50)
fixed_data_z_12_50 = np.array(fixed_data_z_12_50)
fixed_data_z_12_50_over_1000000 = fixed_data_z_12_50/1000000

N_full = np.array(N_full)
Z_full = np.array(Z_full)
fixed_data_full = np.array(fixed_data_full)
fixed_data_full_over_1000000 = fixed_data_full/1000000

N_all_z_12 = np.array(N_all_z_12)
Z_all_z_12 = np.array(Z_all_z_12)
fixed_data_all_z_12 = np.array(fixed_data_all_z_12)
fixed_data_all_z_12_over_1000000 = fixed_data_all_z_12/1000000


In [4]:
@np.vectorize
def valence(x):
    magic = [2,8,20,28,50,82,126,184]
    if x in magic:
        return 0
    else:
        residual = x
        for i in magic:
            if np.abs(i-x) < np.abs(residual):
                residual = np.abs(i-x)
        return residual

v_ns = {}
for n in N:
    v_ns[n] = valence(n)
v_zs = {}
for z in Z:
    v_zs[z] = valence(z)

v_ns_z_12_50 = {}
for n in N_z_12_50:
    v_ns_z_12_50[n] = valence(n)
v_zs_z_12_50 = {}
for z in Z_z_12_50:
    v_zs_z_12_50[z] = valence(z)

v_ns_full = {}
for n in N_full:
    v_ns_full[n] = valence(n)
v_zs_full = {}
for z in Z_full:
    v_zs_full[z] = valence(z)

v_ns_all_z_12 = {}
for n in N_all_z_12:
    v_ns_all_z_12[n] = valence(n)
v_zs_all_z_12 = {}
for z in Z_all_z_12:
    v_zs_all_z_12[z] = valence(z)

In [5]:
@np.vectorize
def pairing(Z, N, aP):
    A = Z + N
    delta = aP * A ** (-1 / 2)
    # delta[(Z % 2 == 1) & (N % 2 == 1)] *= -1
    if (Z % 2 == 1) and (N % 2 == 1):
        delta *= -1
    # delta[A % 1 == 1] = 0
    if (A % 1 == 1):
        delta = 0
    return delta

def volume(Z, N, aV):
    A = Z + N
    return aV * A

def surface(Z, N, aS):
    A = Z + N
    return aS * A ** (2 / 3)

def coulomb(Z, N, aC):
    A = Z + N
    return aC * Z * (Z - 1) / (A ** (1 / 3))

def asymmetry(Z, N, aA):
    A = Z + N
    return aA * (N - Z) ** 2 / A

def rms(one, two):
    return np.sqrt(np.mean((one - two) ** 2))

def shell_helper(Z, N, aN, c1n, c2n, aZ, c1z, c2z, c3n, c3z, c4n, c4z):
    x_n = v_ns_all_z_12[N] - aN
    x_z = v_zs_all_z_12[Z] - aZ
    res = c1n*x_n + c2n*x_n**2 + c3n*x_n**3 + c4n*x_n**4
    res += c1z*x_z + c2z*x_z**2 + c3z*x_z**3 + c4z*x_z**4
    return res

def shifted_piecewise_piecewise_shell(Z, N, args):
    a1z, a2z, a3z, a4z, a1n, a2n, a3n, a4n, b1z, b2z, b3z, b4z, b1n, b2n, b3n, b4n, c1z, c2z, c3z, c4z, c1n, c2n, c3n, c4n, d1z, d2z, d3z, d4z, d1n, d2n, d3n, d4n, e1n, e2n, e3n, e4n, aN1, aN2, aN3, aN4, aN5, aZ1, aZ2, aZ3, aZ4 = args
    res = []
    for n,z in zip(N, Z):
        args = []
        if n<28:
            args.extend([aN1, a1n, a2n, a3n, a4n])
        elif n<50:
            args.extend([aN2, b1n, b2n, b3n, b4n])
        elif n<82:
            args.extend([aN3, c1n, c2n, c3n, c4n])
        elif n<126:
            args.extend([aN4, d1n, d2n, d3n, d4n])
        else:
            args.extend([aN5, e1n, e2n, e3n, e4n])

        if z<28:
            args.extend([aZ1, a1z, a2z, a3z, a4z])
        elif z<50:
            args.extend([aZ2, b1z, b2z, b3z, b4z])
        elif z<82:
            args.extend([aZ3, c1z, c2z, c3z, c4z])
        else:
            args.extend([aZ4, d1z, d2z, d3z, d4z])
        res.append(shell_helper(z, n, *args))
    return res

def double_power_law(x, args):
    a1, b1, c1, d1, a2, b2, c2, d2 = args
    x = np.float64(x)
    b1 = np.clip(b1, -1000, 1000)  # Example range, adjust as needed
    b2 = np.clip(b2, -1000, 1000)  # Example range, adjust as needed

    c1 = np.clip(c1, -10, 10)  # Example range, adjust as needed
    c2 = np.clip(c2, -10, 10)  # Example range, adjust as needed
    return a1 * np.sign(x-b1) * (np.abs(x-b1)) ** c1 + d1 + a2 * np.sign(x-b2) * (np.abs(x-b2)) ** c2 + d2

def fit_pc2_double_power_law_exp_diff(X, args):
    a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3 = args
    Z, N = X
    b1 = np.clip(b1, -1000, 1000)  # Example range, adjust as needed
    b2 = np.clip(b2, -1000, 1000)  # Example range, adjust as needed

    c1 = np.clip(c1, -10, 10)  # Example range, adjust as needed
    c2 = np.clip(c2, -10, 10)  # Example range, adjust as needed
    dbl_pwr_law = a1 * np.sign(Z-b1) * (np.abs(Z-b1)) ** c1 + d1 + a2 * np.sign(Z-b2) * (np.abs(Z-b2)) ** c2 + d2
    exp_i = a3 * np.exp(b3 * (N-Z-c3)) + d3
    return dbl_pwr_law + exp_i


In [6]:
def magic_nums_exp(N, Z, args):
    a1, b1, c1, a2, b2, c2 = args
    d1 = np.sqrt((N-82)**2 + (Z-50)**2)
    d2 = np.sqrt((N-126)**2 + (Z-82)**2)
    res = a1*np.exp(-b1*d1 + c1)
    res += a2*np.exp(-b2*d2 + c2)
    return res

@np.vectorize
def ws3_pairing(N, Z, aP2):
    A = N+Z
    res = aP2*A**(-1/3)
    I = np.abs(N-Z)/A
    if (N%2)==0 and (Z%2)==0:
        res *= 2-I
    elif (N%2)==1 and (Z%2)==1:
        res *= I
    elif (N%2)==0 and (Z%2)==1 and N>Z:
        res *= 1-I
    elif (N%2)==1 and (Z%2)==0 and N<Z:
        res *= 1-I
    return res

def terms_so_far_modified(X, *args):
    Z, N = X
    res = 0
    res += volume(Z, N, args[0])
    res += surface(Z, N, args[1])
    res += coulomb(Z, N, args[2])
    res += asymmetry(Z, N, args[3])
    res += ws3_pairing(Z, N, args[4])
    res += shifted_piecewise_piecewise_shell(Z, N, args[5:50])
    
    T = N-Z
    res += args[50]*T**4 + args[51]*T**3 + args[52]*T**2 + args[53]*T + args[54]
    
    pc1_m_fit = double_power_law(Z, args[55:63])
    pc1_b_fit = double_power_law(Z, args[63:71])
    res += pc1_m_fit*T + pc1_b_fit
    
    res += fit_pc2_double_power_law_exp_diff(X, args[71:83])
    
    res += magic_nums_exp(N, Z, args[83:89])

    return res


In [7]:
def linear_fit(x, m, b):
    return m*x + b

def line_exp(N, Z, args):
    m, b, pitch, shift = args
    res = []
    for n, z in zip(N, Z):
        if z<28:
            pt_on_line = (n+z)/2
            max_val = linear_fit(pt_on_line, m, b)
            dist = np.sqrt((n-pt_on_line)**2 + (z-pt_on_line)**2)
            res.append(max_val*np.exp(-pitch*dist + shift))
        else:
            res.append(0)
    return res

def vector_help(Z, N, cN, cZ):
    iZ = Z - 12
    iN = N - 7
    return cN[int(iN)] + cZ[int(iZ)]


In [8]:
def const_exp(N, Z, args):
    c, pitch, shift = args
    res = []
    for n, z in zip(N, Z):
        if z>27 and z<39:
            pt_on_line = (n+z)/2
            dist = np.sqrt((n-pt_on_line)**2 + (z-pt_on_line)**2)
            res.append(c*np.exp(-pitch*dist + shift))
        else:
            res.append(0)
    return res

def terms_so_far_modified_for_micro_plus_line_plus_const_exp(X, *args):
    args = args[0]
    
    Z, N = X
    res = 0
    res += volume(Z, N, args[0])
    res += surface(Z, N, args[1])
    res += coulomb(Z, N, args[2])
    res += asymmetry(Z, N, args[3])
    res += ws3_pairing(Z, N, args[4])
    res += shifted_piecewise_piecewise_shell(Z, N, args[5:50])
    
    T = N-Z
    res += args[50]*T**4 + args[51]*T**3 + args[52]*T**2 + args[53]*T + args[54]
    
    pc1_m_fit = double_power_law(Z, args[55:63])
    pc1_b_fit = double_power_law(Z, args[63:71])
    res += pc1_m_fit*T + pc1_b_fit
    
    res += fit_pc2_double_power_law_exp_diff(X, args[71:83])
    
    res += magic_nums_exp(N, Z, args[83:89])
    res += line_exp(N, Z, args[89:93])
    res += const_exp(N, Z, args[93:96])

    return res

def microcorrections_plus_exp_line(X, *args):
    Z, N = X
    vg = np.vectorize(vector_help, excluded=['cN','cZ'])
    cZ = args[96:195]
    cN = args[195:]
    return vg(Z=Z, N=N, cZ=cZ, cN=cN)+terms_so_far_modified_for_micro_plus_line_plus_const_exp(X, args[:96])


In [9]:
p0 = [ 2.71539328e-02, -3.23225686e-03, -8.10686686e-04, -1.81003616e-02,
  6.28303444e-03, -4.26599542e-07,  8.13408184e-08,  1.61217587e-08,
 -6.93200304e-11,  1.03388614e-02, -1.62057321e-04,  1.21021440e+02,
  4.62718761e-03, -6.17995846e-07,  8.36800896e-08,  1.77654894e-08,
 -1.37582808e-10,  4.00997036e-02, -4.08376577e-04, -1.17068753e+02,
 -1.42826076e-02,  2.42167887e-07, -2.67825021e-06,  1.11864355e-07,
 -1.15515277e-07, -5.24017172e-04,  3.87683999e-05,  9.54748756e+00,
 -1.97546913e-04,  2.87340822e-07,  1.06802328e-05,  3.47773059e-08,
  4.59818853e-07, -2.31919492e-04,  3.43583723e-05,  1.03610198e+01,
  9.87604048e-05, -1.47210653e-03,  6.36784685e-05,  1.37292637e+01,
  8.50284766e-04, -6.36341007e+01, -8.03590036e+01,  1.58092100e+01,
  1.17704351e+01, -5.30876820e+00,  4.00508637e-05,  4.37960341e-05,
  1.20576382e-04,  1.96776248e-04, -3.22842864e-08, -9.29491137e-07,
 -2.55339190e-04, -1.01770276e-02, -1.73596177e-01, -8.23527492e-05,
  1.29224245e+01,  3.26163124e-01, -1.01515295e-02,  8.76315512e-07,
  2.23429835e+01,  1.04192172e+00, -1.01655830e-02, -6.96061113e-02,
 -2.30707958e+00,  7.90650196e-01, -1.76172078e-01, -1.01841256e-02,
  1.74283514e+01,  9.27052780e-01, -1.74666582e-01, -3.44098646e-03,
  1.20097419e+01,  6.61873254e-01, -1.76209380e-01,  2.00091360e-01,
 -4.73677248e+00,  3.78483190e-01, -1.74088984e-01,  2.10983555e-01,
  2.66728987e-02, -4.07051052e+01, -1.75308726e-01,  1.20667364e-01,
  1.63006839e-01, -2.57924481e+00,  1.69816792e-02,  1.07052087e-01,
 -2.64856534e-01,  2.49625164e-04, -6.20525419e-02, -4.96183063e-03,
  9.76605058e-01,  2.52909929e-02,  3.85863561e-03, -3.68291808e-01,
  8.81188100e-02,  9.20067574e-02,  9.50490250e-02,  9.55299002e-02,
  9.54939605e-02,  9.62348785e-02,  9.78746598e-02,  9.87419381e-02,
  9.89170433e-02,  1.00476580e-01,  1.02177398e-01,  1.04504419e-01,
  1.06781367e-01,  1.06871999e-01,  1.06322119e-01,  1.05801549e-01,
 -7.07215271e-02, -6.84013575e-02, -6.67239162e-02, -6.45834573e-02,
 -6.29766252e-02, -6.12593078e-02, -5.96572677e-02, -5.81008328e-02,
 -5.64345619e-02, -5.49981530e-02, -5.33459097e-02, -3.49119392e-02,
 -3.39503051e-02, -3.31249575e-02, -3.21348985e-02, -3.12014423e-02,
 -3.01014889e-02, -2.91469339e-02, -2.83004843e-02, -2.73529301e-02,
 -2.64543568e-02, -2.55160891e-02, -3.32691017e-02, -3.00653646e-02,
 -2.72071953e-02, -2.43522842e-02, -2.17370205e-02, -1.93293841e-02,
 -1.70165626e-02, -1.50351597e-02, -1.33279482e-02, -1.19669856e-02,
 -1.06563674e-02, -9.64262215e-03, -8.89307428e-03, -8.24337108e-03,
 -7.85897844e-03, -7.60892432e-03, -7.60690403e-03, -5.53689591e-03,
 -3.50556983e-03, -1.69618520e-03, -4.59336390e-09,  1.36162800e-03,
  2.31956949e-03,  3.27974410e-03,  3.77329185e-03,  4.13119307e-03,
  4.02074109e-03,  3.68556221e-03,  2.99451455e-03,  2.27067171e-03,
  1.17709454e-03, -1.30063347e-04, -1.77993457e-03,  9.97369110e-04,
  3.49947129e-03,  5.77883556e-03,  7.96155058e-03,  9.96019201e-03,
  1.18133666e-02,  1.34276821e-02,  1.48957418e-02,  1.61824585e-02,
  1.72904782e-02,  1.83041126e-02,  1.88694124e-02,  1.93218234e-02,
  1.91460894e-02,  1.85771149e-02,  1.74726055e-02,  1.58622249e-02,
  1.32803415e-02,  1.02952820e-02,  5.84203226e-03,  9.02487813e-04,
 -5.55037215e-03,  1.91049629e-03,  7.75529090e-03,  1.29426999e-02,
  1.67389829e-02,  2.01249801e-02,  2.20596738e-02,  3.11644005e-02,
  2.85839797e-02,  2.30049798e-02,  1.94591652e-02,  1.56351405e-02,
  1.20919244e-02,  7.83984284e-03,  4.11806148e-03,  5.99393692e-03,
  6.73707865e-03,  7.67109842e-03,  7.82649812e-03,  8.24026052e-03,
  8.16813391e-03,  5.06092581e-03,  2.35336696e-03, -5.61027673e-04,
 -3.32029683e-03, -1.06280457e-03,  6.87158020e-04,  2.42399072e-03,
 -1.09590494e-03, -1.20256782e-03, -2.58543003e-03, -3.58252942e-03,
 -5.34418816e-03, -7.14257164e-03, -9.85887855e-03, -1.26702323e-02,
 -1.60036102e-02, -1.93883719e-02, -2.32746005e-02, -2.77931675e-02,
 -2.44831034e-02, -2.18091249e-02, -1.93402990e-02, -1.74981610e-02,
 -1.57152838e-02, -1.45690843e-02, -1.34140516e-02, -1.27157200e-02,
 -1.21189218e-02, -1.19003166e-02,  1.00934852e-03,  2.58158334e-03,
  3.45313248e-03,  4.65508607e-03,  5.28298773e-03,  6.19001252e-03,
  6.64650387e-03,  7.15209685e-03,  7.36638177e-03,  7.62522974e-03,
  8.15716132e-03,  8.50464462e-03,  8.90955519e-03,  9.07058860e-03,
  9.39083314e-03,  9.47346924e-03,  9.80009421e-03,  8.68263063e-03,
  7.96269943e-03,  6.76935820e-03,  5.96370580e-03,  4.69988784e-03,
  3.72982413e-03,  2.28637833e-03,  1.16277670e-03, -4.21383397e-04,
 -1.75181599e-03, -3.47188002e-03, -5.20433338e-03, -7.29458241e-03,
 -9.23986635e-03, -1.15393643e-02,  6.24003075e-04,  1.26442089e-03,
  1.62232283e-03,  2.13502163e-03,  2.45414311e-03,  2.62030771e-03,
  3.00301364e-03,  3.06321514e-03,  3.65168910e-03,  3.80561631e-03,
  4.40230944e-03,  4.53648589e-03,  5.02390484e-03,  5.14732307e-03,
  5.33774292e-03,  5.26217603e-03,  5.23334291e-03,  5.03395364e-03,
  4.74075379e-03,  4.34631797e-03,  3.76075104e-03,  3.01792550e-03,
  2.10045431e-03,  2.88908074e-03,  3.66982865e-03,  4.13637284e-03,
  4.71625245e-03,  4.84661723e-03,  5.18312643e-03,  5.16524255e-03,
  5.34150973e-03,  5.27644897e-03,  5.37524083e-03,  5.38504233e-03,
  5.38341260e-03,  5.13809271e-03,  5.11524571e-03,  4.83516059e-03,
  4.49873143e-03,  4.10130505e-03,  3.56219531e-03,  3.02504118e-03,
  2.35741387e-03,  1.88937568e-03,  1.51357702e-02,  1.53885373e-02,
  1.57240633e-02,  1.61164974e-02,  1.64854290e-02,  1.66766837e-02,
  1.69832651e-02,  1.71665655e-02,  1.70636336e-02,  1.68528441e-02,
  1.65429220e-02,  1.59734035e-02,  1.53131041e-02,  1.43008266e-02,
  1.31442660e-02,  1.16504969e-02,  9.88821682e-03,  7.75384697e-03,
  5.19647821e-03,  2.41164335e-03, -1.08704626e-03, -4.87206627e-03,
 -9.42145977e-03, -1.39786661e-02, -1.93833046e-02, -2.49828136e-02,
 -3.14346200e-02, -3.84506290e-02, -4.63478309e-02, -5.46411702e-02,
 -4.47156620e-02, -3.52005719e-02, -2.65063427e-02, -1.81694069e-02,
 -1.04046887e-02, -2.87248728e-03]

# micro_popt_plus_line_plus_const, pcov = curve_fit(microcorrections_plus_exp_line, (Z_all_z_12,N_all_z_12), fixed_data_all_z_12_over_1000000, p0, maxfev=10000000)
# print(micro_popt_plus_line_plus_const)
# res = microcorrections_plus_exp_line((Z_all_z_12,N_all_z_12),*micro_popt_plus_line_plus_const)

# rms(fixed_data_all_z_12_over_1000000, res)*1000


In [10]:
preds = []

print(len(fixed_data_all_z_12_over_1000000))
temp_n = []
temp_z = []
temp_d = []
temp_correct_be = 0
for n, z, d in zip(full_features_N,full_features_Z,ouail_data):
    if (n,z) in data_all_z_12.keys():
        if n != remove_n and z != remove_z:
            temp_n.append(n)
            temp_z.append(z)
            temp_d.append(d/1000000) 
        else:
            temp_correct_be = d/1000000
temp_popt, pcov = curve_fit(microcorrections_plus_exp_line, (temp_z,temp_n), temp_d, p0, maxfev=10000000)
res = microcorrections_plus_exp_line((np.array([remove_z]),np.array([remove_n])),*temp_popt)

with open('/home/submit/karich/output.txt', 'a') as file:
    file.write(f'{remove_n}, {remove_z}, {res}\n')

2334


/tmp/ipykernel_2245373/1769200164.py:5: RuntimeWarning: overflow encountered in exp
  res = a1*np.exp(-b1*d1 + c1)


/tmp/ipykernel_2245373/1301862624.py:16: OptimizeWarning: Covariance of the parameters could not be estimated
  temp_popt, pcov = curve_fit(microcorrections_plus_exp_line, (temp_z,temp_n), temp_d, p0, maxfev=10000000)
/tmp/ipykernel_2245373/3639107253.py:68: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x = np.float64(x)
